<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/4_1_activelearning_ordinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Active Learning with Ordinal encoding data
1. Ordinal encoding categorical fileds
2. Process the labeled fraud invoice
3. Sample data for active learning
4. Train model
5. Choose top 100 uncertain invoice
6. Get back the labeled data

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, roc_auc_score

In [3]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [4]:
df = df.drop(columns='Unnamed: 0')

In [5]:
df.tail()

,distributor,sales,branch,inv_type,invoice_no,product_no,prod_cla,qty,total_amt,sale_price,ship_qty,cust_type,return,discount_app,list_price,inv_ship_days
228596,A15020494,PJ036429,720018,Z038,2111414500,BFD37B45,11020.0,1,83.59,83.59,1,AA02,0,1,94.46,3
228597,A15020494,PJ036429,720018,Z038,2111414500,BRC1H611,11020.0,1,417.95,417.95,1,AA02,0,1,472.28,3
228598,A15020494,PJ036429,720018,Z038,2111414500,FPDAP28BAP,11020.0,1,4188.73,4188.73,1,AA02,0,1,4733.26,3
228599,A15050131,PJ040864,720037,Z038,2111414525,BRC301B611,11050.0,1,398.23,398.23,1,AB02,0,1,450.00,4
228600,A15050131,PJ040864,720037,Z038,2111414525,VAML350HV1,11050.0,1,7654.87,7654.87,1,AB02,0,1,8650.00,4


## import label data and sampling

In [6]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [7]:
file = path + 'frauds2.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [8]:
df_fraud

,invoice_no
0,2111304677
1,2111329548
2,2111336900
3,2111349984
4,2111359806
5,2111368995
6,2111369060
7,2111373597
8,2111393180
9,2111400912


In [9]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [10]:
df = label_fraud(df, df_fraud,df_fraud2)

In [11]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Encoding

In [12]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [13]:
def field_encoding(df):
  '''
  Ordinal encode categorical fields.
  '''
  cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']
  df[cols] = df[cols].astype('str') 
  enc = LabelEncoder()
  for f in cols:
    df[f] = enc.fit_transform(df[f])
  return df

In [14]:
df_enc = field_encoding(df)

In [15]:
df_enc.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
fraud                3
dtype: int64

In [16]:
df_enc.head()

,distributor,sales,branch,inv_type,invoice_no,product_no,prod_cla,qty,total_amt,sale_price,ship_qty,cust_type,return,discount_app,list_price,inv_ship_days,fraud
0,637,39,0,0,0,513,6,1,1000.00,1000.00,0,0,0,0,1000.0,239,0.0
1,450,21,10,1,1,984,0,4,130973.44,32743.36,0,8,1,1,37000.0,237,0.0
2,118,110,23,1,2,57,1,1,398.23,398.23,0,0,1,1,450.0,225,0.0
3,118,110,23,1,2,552,0,1,2831.86,2831.86,0,0,1,1,3200.0,225,0.0
4,68,142,23,1,3,57,1,4,1592.92,398.23,0,0,1,1,450.0,225,0.0


## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [17]:
df_fraud = df_enc[df_enc['fraud'] == 1]
df_non_fraud = df_enc[df_enc['fraud'] == 0]
df_fraud2 = df_enc[df_enc['fraud'] == 2]

In [18]:
df_fraud2.tail()

,distributor,sales,branch,inv_type,invoice_no,product_no,prod_cla,qty,total_amt,sale_price,ship_qty,cust_type,return,discount_app,list_price,inv_ship_days,fraud
217821,483,87,2,2,29200,252,1,2,6337.18,2883.4169,2,10,0,1,3580.5,0,2.0
217822,483,87,2,2,29200,57,1,2,613.28,279.0424,2,10,0,1,346.5,0,2.0
217823,483,87,2,2,29200,256,1,2,6555.22,2982.6251,2,10,0,1,3703.7,0,2.0
217824,483,87,2,2,29200,57,1,1,306.64,279.0424,1,10,0,1,346.5,0,2.0
217825,483,87,2,2,29200,265,1,1,3815.93,3472.4963,1,10,0,1,4312.0,0,2.0


In [19]:
def sampling_inv(df_non_fraud):
  '''
  Random sampling from unlabled invoices.
  '''
  sample_inv = df_non_fraud['invoice_no'].unique()
  print(len(sample_inv))
  sample_inv1 = np.random.choice(sample_inv,size=1000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv1)
  print(len(sample_inv))
  sample_inv2 = np.random.choice(sample_inv,size=10000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv2)
  print(len(sample_inv))
  return sample_inv1, sample_inv2, sample_inv


In [20]:
sample_inv1, sample_inv2, sample_inv = sampling_inv(df_non_fraud)

30686
29686
19686


In [21]:
len(np.unique(sample_inv2))

10000

In [22]:
## Build training dataset
df_train = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv1)]
df_train = pd.concat([df_train,df_fraud])
df_train = df_train.sort_index()

In [49]:
## Build eval dataset
df_eval = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv2)]
df_eval = pd.concat([df_eval,df_fraud2])
df_eval.loc[df_eval['fraud'] == 2, 'fraud'] = 1 
df_eval = df_eval.sort_index()

In [ ]:
df_train.head()

## Standardscale data

In [30]:
scaler = StandardScaler()

In [31]:
X_train = df_train.drop(columns=['fraud'])
y_train = df_train['fraud'].values
X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [32]:
X_eval = df_eval.drop(columns=['fraud'])
y_eval = df_eval['fraud'].values
X_eval = scaler.transform(X_eval)
y_eval = y_eval.reshape(len(y_eval),1)

In [33]:
X_train[0]

array([ 1.12170975, -0.0171135 ,  1.65998078,  0.03678559, -1.75726635,
        1.5801828 , -0.08821103, -0.09347048,  0.09596717,  1.36282573,
       -0.09145284, -0.23992399, -0.03678559,  0.        ,  1.3610212 ,
       -0.16311424])

In [34]:
y_eval.shape

(74868, 1)

## Build model

In [35]:
# define the  model
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,089
Trainable params: 1,089
Non-trainable params: 0
_________________________________________________________________


## Training model

In [37]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
232/232 [==============================] - 1s 2ms/step - loss: 0.3367 - accuracy: 0.8600
Epoch 2/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0963 - accuracy: 0.9792
Epoch 3/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0868 - accuracy: 0.9795
Epoch 4/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0819 - accuracy: 0.9818
Epoch 5/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0777 - accuracy: 0.9823
Epoch 6/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0740 - accuracy: 0.9823
Epoch 7/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0699 - accuracy: 0.9828
Epoch 8/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0650 - accuracy: 0.9832
Epoch 9/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0611 - accuracy: 0.9830
Epoch 10/100
232/232 [==============================] - 0s 2ms/step - loss: 0.0565 - accura

In [38]:
loss,accuracy = model.evaluate(X_eval, y_eval)
print(loss, accuracy)

2340/2340 [==============================] - 2s 963us/step - loss: 0.1526 - accuracy: 0.9866
0.15255092084407806 0.9865897297859192


## Predict on eval dataset for further labeling.

In [39]:
y_pred = model.predict(X_eval)
y_cls = (y_pred > 0.5).astype("int32")

In [60]:
(y_pred > 0.5).sum()

827

In [40]:
def eval_model(y_test, y_cls):
  y1 = y_eval.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))


In [ ]:
df_eval.head(100)

In [61]:
def record_to_label(df_eval,y_pred):
  df_eval['predict'] = y_pred
  df_eval['mid_value'] = abs(df_eval['predict'] - 0.5)
  df_eval = df_eval.sort_values('mid_value')
  value_limit = df_eval.iloc[100,-1]
  df_record4label = df_eval[(df_eval['mid_value'] <= value_limit)|(df_eval['fraud'] == 1)]
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'for_label.csv'
  df_record4label.to_csv(file_csv)
  return True


In [62]:
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'for_label.csv'
  df_record4label.to_csv(file_csv)